In [1]:
import os
import random
from pathlib import Path
from PIL import Image
from ultralytics import YOLO

In [2]:
def create_yolo_dataset(src_root, out_root, split=(0.8, 0.1, 0.1)):
    src_root = Path(src_root)
    out_root = Path(out_root)

    # Ensure output directory exists
    if not out_root.exists():
        out_root.mkdir(parents=True, exist_ok=True)

    # Create YOLO folder structure
    for sub in ["images/train", "images/val", "images/test",
                "labels/train", "labels/val", "labels/test"]:
        (out_root / sub).mkdir(parents=True, exist_ok=True)

    # Get classes
    classes = sorted([d.name for d in src_root.iterdir() if d.is_dir()])
    class_to_id = {c: i for i, c in enumerate(classes)}

    # Collect images
    images, labels = [], []
    for cls in classes:
        for img_path in (src_root / cls).glob("*.*"):
            if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
                continue
            images.append(img_path)
            labels.append(class_to_id[cls])

    # Shuffle dataset
    combined = list(zip(images, labels))
    random.seed(42)
    random.shuffle(combined)
    if combined:
        images, labels = zip(*combined)
    else:
        images, labels = [], []

    n = len(images)
    n_train = max(1, int(n * split[0]))
    n_val = max(1, int(n * split[1]))
    n_test = max(0, n - n_train - n_val)

    train_imgs, train_lbls = images[:n_train], labels[:n_train]
    val_imgs, val_lbls = images[n_train:n_train+n_val], labels[n_train:n_train+n_val]
    test_imgs, test_lbls = images[n_train+n_val:], labels[n_train+n_val:]

    splits = {
        "train": (train_imgs, train_lbls),
        "val": (val_imgs, val_lbls),
        "test": (test_imgs, test_lbls)
    }

    # Save images and labels
    for sp, (img_list, lbl_list) in splits.items():
        for img_path, lbl in zip(img_list, lbl_list):
            img = Image.open(img_path)
            if img.mode == "RGBA":
                img = img.convert("RGB")

            dst_img_path = out_root / f"images/{sp}/{img_path.stem}.jpg"
            img.save(dst_img_path, format="JPEG", quality=95)

            # YOLO label (dummy full-image box)
            with open(out_root / f"labels/{sp}/{img_path.stem}.txt", "w") as f:
                f.write(f"{lbl} 0.5 0.5 1.0 1.0\n")

    # Absolute paths for data.yaml
    abs_path = out_root.resolve()
    yaml_text = f"""
train: {abs_path}/images/train
val: {abs_path}/images/val
test: {abs_path}/images/test

nc: {len(classes)}
names: {classes}
"""
    with open(out_root / "data.yaml", "w") as f:
        f.write(yaml_text)

    print("✅ YOLO dataset ready at:", out_root)
    print("📌 Classes:", classes)


In [3]:
ROOT_DIR = r"C:\Users\HP\OneDrive\Documents\Project\Project 5\Solar Panel"
YOLO_DATA_ROOT = "dataset_yolo"

create_yolo_dataset(ROOT_DIR, YOLO_DATA_ROOT, split=(0.8, 0.1, 0.1))

# Verify dataset
base = Path(YOLO_DATA_ROOT)
print("Train images:", len(list((base/"images/train").glob("*"))))
print("Val images:", len(list((base/"images/val").glob("*"))))
print("Test images:", len(list((base/"images/test").glob("*"))))
print("data.yaml exists:", (base/"data.yaml").exists())

✅ YOLO dataset ready at: dataset_yolo
📌 Classes: ['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']
Train images: 694
Val images: 86
Test images: 88
data.yaml exists: True


In [4]:
model = YOLO("yolov8n.pt")  # small YOLOv8 model

model.train(
    data=str(Path(YOLO_DATA_ROOT)/"data.yaml"),
    epochs=30,
    imgsz=640,
    project="runs/yolo",
    name="yolov8n_autobox",
    exist_ok=True
)

New https://pypi.org/project/ultralytics/8.3.194 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.193  Python-3.13.2 torch-2.7.1+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_yolo\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_autobox, nbs=64, nms=False, opset=None, optim

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000222BFEA0B40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
 

In [5]:
results = model.predict(
    source=str(Path(YOLO_DATA_ROOT)/"images/test"),
    save=True,
    imgsz=640,
    conf=0.25
)


image 1/88 C:\Users\HP\OneDrive\Documents\Project\Project 5\dataset_yolo\images\test\Bird (104).jpg: 480x640 1 Bird-drop, 228.8ms
image 2/88 C:\Users\HP\OneDrive\Documents\Project\Project 5\dataset_yolo\images\test\Bird (121).jpg: 384x640 1 Bird-drop, 143.0ms
image 3/88 C:\Users\HP\OneDrive\Documents\Project\Project 5\dataset_yolo\images\test\Bird (123).jpg: 640x416 1 Bird-drop, 156.1ms
image 4/88 C:\Users\HP\OneDrive\Documents\Project\Project 5\dataset_yolo\images\test\Bird (126).jpg: 480x640 1 Bird-drop, 140.3ms
image 5/88 C:\Users\HP\OneDrive\Documents\Project\Project 5\dataset_yolo\images\test\Bird (128).jpg: 384x640 1 Bird-drop, 127.8ms
image 6/88 C:\Users\HP\OneDrive\Documents\Project\Project 5\dataset_yolo\images\test\Bird (139).jpg: 480x640 1 Bird-drop, 134.3ms
image 7/88 C:\Users\HP\OneDrive\Documents\Project\Project 5\dataset_yolo\images\test\Bird (140).jpg: 480x640 1 Bird-drop, 128.9ms
image 8/88 C:\Users\HP\OneDrive\Documents\Project\Project 5\dataset_yolo\images\test\Bird

In [8]:
# Validate on validation set
obj_eval = model.val()  # automatically uses val set from data.yaml

# Metrics
# Metrics
print("mAP@0.5:", obj_eval.box.map50)   # mAP@0.5
print("mAP@0.5:0.95:", obj_eval.box.map)  # mAP@0.5:0.95
print("Precision:", obj_eval.box.mp)     # Mean Precision
print("Recall:", obj_eval.box.mr)        # Mean Recall
print("F1 Score:", obj_eval.box.f1)           # F1 Score

Ultralytics 8.3.193  Python-3.13.2 torch-2.7.1+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
val: Fast image access  (ping: 0.91.1 ms, read: 245.7161.7 MB/s, size: 132.4 KB)
val: Scanning C:\Users\HP\OneDrive\Documents\Project\Project 5\dataset_yolo\labels\val.cache... 86 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 86/86 60.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 0.7it/s 8.7s2.0ss
                   all         86         86      0.887      0.945      0.954      0.954
             Bird-drop         15         15      0.787      0.933      0.903      0.903
                 Clean         15         15       0.82      0.933      0.954      0.954
                 Dusty         19         19      0.928          1       0.98       0.98
     Electrical-damage         11         11      0.915      0.978      0.988      0.988
       Physical-Damage          8          8          1      0.935      0.995     

In [9]:
# Number of classes
nc = obj_eval.box.nc  

for i in range(nc):
    p, r, ap50, ap = obj_eval.box.class_result(i)
    print(f"Class {i}: Precision={p:.3f}, Recall={r:.3f}, AP@0.5={ap50:.3f}, AP@0.5:0.95={ap:.3f}")

Class 0: Precision=0.787, Recall=0.933, AP@0.5=0.903, AP@0.5:0.95=0.903
Class 1: Precision=0.820, Recall=0.933, AP@0.5=0.954, AP@0.5:0.95=0.954
Class 2: Precision=0.928, Recall=1.000, AP@0.5=0.980, AP@0.5:0.95=0.980
Class 3: Precision=0.915, Recall=0.978, AP@0.5=0.988, AP@0.5:0.95=0.988
Class 4: Precision=1.000, Recall=0.935, AP@0.5=0.995, AP@0.5:0.95=0.995
Class 5: Precision=0.875, Recall=0.889, AP@0.5=0.905, AP@0.5:0.95=0.905


In [11]:
from ultralytics import YOLO

# Load your trained model (best is usually preferred)
model = YOLO(r"C:\Users\HP\OneDrive\Documents\Project\Project 5\runs\yolo\yolov8n_autobox\weights\best.pt")

# Run prediction on a single image
results = model.predict(r"C:\Users\HP\OneDrive\Documents\Project\Project 5\test_images\sample.jpeg", save=True, conf=0.25)



image 1/1 C:\Users\HP\OneDrive\Documents\Project\Project 5\test_images\sample.jpeg: 640x640 1 Clean, 240.2ms
Speed: 9.5ms preprocess, 240.2ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\HP\OneDrive\Documents\Project\Project 5\runs\detect\predict
